In [1]:
!pip install tensorflow


In [2]:
# 📌 Step 1: Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, classification_report, confusion_matrix
)

# 📂 Step 2: Load and Clean Dataset
df = pd.read_csv('Sleep_Data_Sampled(missing_Dis).csv')
df.drop(columns=['Person ID'], inplace=True)

# Label Encode Categorical Columns
categorical_cols = ['Gender', 'Occupation', 'BMI Category', 'Blood Pressure']
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

# 🎯 Step 3: Define Features and Targets
X = df.drop(columns=['Quality of Sleep'])
y_reg = df['Quality of Sleep']
y_class = y_reg.apply(lambda x: 0 if x <= 4 else (1 if x <= 7 else 2))  # 0: Poor, 1: Average, 2: Good

# ⚖ Step 4: Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 🔀 Step 5: Train/Test Split
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_scaled, y_reg, test_size=0.2, random_state=42)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_scaled, y_class, test_size=0.2, random_state=42)

# Convert classification labels to categorical (one-hot)
y_train_c_cat = tf.keras.utils.to_categorical(y_train_c, num_classes=3)
y_test_c_cat = tf.keras.utils.to_categorical(y_test_c, num_classes=3)

# 🟢 Step 6A: Keras Regression Model
model_reg = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_r.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Single output for regression
])
model_reg.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_reg.fit(X_train_r, y_train_r, epochs=100, batch_size=16, verbose=0)

# Predict and Evaluate Regression
y_pred_r = model_reg.predict(X_test_r).flatten()
print("📊 Regression Evaluation (Keras)")
print("MAE:", mean_absolute_error(y_test_r, y_pred_r))
print("MSE:", mean_squared_error(y_test_r, y_pred_r))
print("R² Score:", r2_score(y_test_r, y_pred_r))

# 🟢 Step 6B: Keras Classification Model
model_cls = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_c.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes
])
model_cls.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cls.fit(X_train_c, y_train_c_cat, epochs=100, batch_size=16, verbose=0)

# Predict and Evaluate Classification
y_pred_probs = model_cls.predict(X_test_c)
y_pred_c = np.argmax(y_pred_probs, axis=1)

print("\n📊 Classification Evaluation (Keras)")
print("Accuracy:", accuracy_score(y_test_c, y_pred_c))
print("Confusion Matrix:\n", confusion_matrix(y_test_c, y_pred_c))
print("Classification Report:\n", classification_report(y_test_c, y_pred_c, target_names=["Poor", "Average", "Good"]))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
📊 Regression Evaluation (Keras)
MAE: 0.06086233630776405
MSE: 0.015356623567640781
R² Score: 0.9861029386520386


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

📊 Classification Evaluation (Keras)
Accuracy: 0.9956666666666667
Confusion Matrix:
 [[   8    1    0]
 [   1 1607    6]
 [   0    5 1372]]
Classification Report:
               precision    recall  f1-score   support

        Poor       0.89      0.89      0.89         9
     Average       1.00      1.00      1.00      1614
        Good       1.00      1.00      1.00      1377

    accuracy                           1.00      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       1.00      1.00      1.00      3000

